In [2]:
import os
import csv
import pandas as pd
from collections import Counter, OrderedDict
from Bio import SeqIO
from itertools import combinations, product

# Counts

In [9]:
clusters = {}
descriptions = {}

with open('lpp/blast_rodeo/blast_rodeo_out/peptidase_RODEO/main_co_occur.csv', 'r') as i:
    next(i)
    for row in csv.reader(i):
        if len(row) > 1:
            
            if len(row) > 7:
                dom = row[8]
                description = row[9]
            else:
                dom = 'no_match'
                description = 'No matching domains found'
            descriptions[dom] = description
            
            if row[0] not in clusters:
                clusters[row[0]] = []
            clusters[row[0]].append(dom)

In [11]:
clusters

{'WP_026694389.1': ['PS_pyruv_trans',
  'Glycos_transf_1',
  'Glycos_transf_1',
  'TIGR03570',
  'Glyco_hydro_88',
  'Polysacc_synt',
  'Acyl_transf_3',
  'no_match',
  'TIGR04352',
  'PqqD',
  'TIGR02228',
  'TIGR02203',
  'NTP_transf_5',
  'VanZ',
  'no_match',
  'no_match',
  'SinI',
  'HTH_3',
  'TIGR00963',
  'TIGR00020'],
 'WP_077623675.1': ['no_match',
  'Gas_vesicle',
  'GvpK',
  'Gas_vesicle',
  'GvpL_GvpF',
  'GvpG',
  'GvpL_GvpF',
  'TIGR02640',
  'GvpO',
  'Gas_vesicle',
  'no_match',
  'YtxH',
  'Metallophos',
  'NTP_transf_5',
  'TIGR02203',
  'TIGR02228',
  'PqqD',
  'TIGR04249',
  'TIGR02203',
  'NTP_transf_5',
  'Transglut_core3',
  'PqqD',
  'TIGR04352',
  'TIGR01536',
  'Acyl_transf_3'],
 'WP_114383165.1': ['TIGR01228',
  'UvrD_C',
  'TIGR03006',
  'TIGR01934',
  'TIGR00054',
  'ThiF',
  'HATPase_c_5',
  'TIGR04352',
  'PqqD',
  'TIGR02228',
  'TIGR02203',
  'NTP_transf_5',
  'Peptidase_S8',
  'no_match',
  'SBP_bac_3',
  'TIGR03004',
  'TIGR03005',
  'TIGR03693'],
 

In [13]:
def count(clusters):
    flat_clusters = []
    for x in clusters:
        flat_clusters += clusters[x]
    return(Counter(flat_clusters))

In [15]:
def cocount(clusters):
    flat_pairs = []
    for x in clusters:
        domains = set(clusters[x])
        pairs = list(combinations(domains, 2))
        flat_pairs += [tuple(sorted(x)) for x in pairs]
    return(Counter(flat_pairs))

In [17]:
counter = count(clusters)
cocounter = cocount(clusters)

In [19]:
lpp_counter = count(clusters)
lpp_cocounter = cocount(clusters)

In [21]:
df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
df.columns = ['Domain', 'Count']
df = df.sort_values(by='Count', ascending=False)
df['Description'] = df['Domain'].map(descriptions)

In [23]:
df

,Domain,Count,Description
6,no_match,661,No matching domains found
9,TIGR02228,192,sigpep_I_arch: signal peptidase I
8,PqqD,186,Coenzyme PQQ synthesis protein D (PqqD)
11,NTP_transf_5,156,Uncharacterised nucleotidyltransferase
10,TIGR02203,104,MsbA_lipidA: lipid A export permease/ATP-bindi...
...,...,...,...
416,TIGR00280,1,eL43_euk_arch: ribosomal protein eL43
414,TIGR01750,1,fabZ: beta-hydroxyacyl-(acyl-carrier-protein) ...
412,TIGR02488,1,flgG_G_neg: flagellar basal-body rod protein FlgG
411,TIGR02490,1,flgF: flagellar basal-body rod protein FlgF


In [25]:
codf = pd.DataFrame([(k[0], k[1], v) for k, v in cocounter.items()])
codf.columns = ['Domain 1', 'Domain 2', 'Co-occurrence']
codf = codf.sort_values(by='Co-occurrence', ascending=False)

In [27]:
codf

,Domain 1,Domain 2,Co-occurrence
88,TIGR02228,no_match,183
78,PqqD,no_match,143
70,PqqD,TIGR02228,142
84,NTP_transf_5,TIGR02228,120
118,NTP_transf_5,no_match,119
...,...,...,...
5342,Asparaginase_II,PqqD,1
5341,Asparaginase_II,YbjN,1
5340,Asparaginase_II,SNARE_assoc,1
5339,Asparaginase_II,BPD_transp_1,1


In [33]:
with pd.ExcelWriter('counts.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Domain count', index=False)
    codf.to_excel(writer, sheet_name='Domain co-occurrence', index=False)

# Domain clusters

In [36]:
dom_cl = {}
with open('dom_cluster.csv', 'r') as i:
    for row in csv.reader(i, delimiter = ';'):
        dom_cl[row[0]] = [x for x in row[:8] if x != '']

In [38]:
dom_cl

{'TIGR02228': ['TIGR02228'],
 'TIGR04249': ['TIGR04249', 'TIGR04352', 'TIGR04355', 'AAA_29', 'TIGR00679'],
 'PqqD (TIGR03859)': ['PqqD (TIGR03859)'],
 'NTP_transf_5': ['NTP_transf_5'],
 'TIGR02203': ['TIGR02203', 'TIGR02204', 'TIGR03796'],
 'Glycos_transf_1': ['Glycos_transf_1',
  'TIGR03088',
  'Glyco_trans_1_4',
  'TIGR03999'],
 'TIGR01179': ['TIGR01179',
  'GDP_Man_Dehyd',
  'Epimerase',
  'TIGR01099',
  'TIGR01207',
  'TIGR01181',
  'TIGR01214'],
 'TIGR01536': ['TIGR01536', 'TIGR03108'],
 'Glycos_transf_2': ['Glycos_transf_2',
  'TIGR03965',
  'Glyco_tranf_2_3',
  'TIGR04009'],
 'Transglut_core3': ['Transglut_core3'],
 'VanZ': ['VanZ'],
 'Polysacc_synt': ['Polysacc_synt',
  'Polysacc_synt_2',
  'Polysacc_synt_3',
  'Polysacc_synt_C'],
 'Metallophos': ['Metallophos', 'Metallophos_2'],
 'TIGR03740': ['TIGR03740', 'TIGR01188', 'TIGR02211', 'TIGR03522'],
 'TIGR01006': ['TIGR01006', 'Wzz'],
 'BPD_transp_1': ['BPD_transp_1', 'TIGR02789', 'TIGR02790'],
 'TIGR01007': ['TIGR01007'],
 'Acyl_

In [40]:
dom_cl_flat = list(dom_cl.keys())
for x in dom_cl:
    dom_cl_flat += dom_cl[x]
dom_cl_flat = set(dom_cl_flat)

In [42]:
dom_cl_flat

{'AAA_29',
 'ABC2_membrane',
 'ABC2_membrane_2',
 'ABC2_membrane_3',
 'ABC2_membrane_4',
 'Acetyltransf_1',
 'Acetyltransf_3',
 'Acetyltransf_9',
 'Acyl_transf_3',
 'Aldo_ket_red',
 'BPD_transp_1',
 'Beta_helix',
 'Cupin_2',
 'DUF1854',
 'DZR',
 'Epimerase',
 'GDP_Man_Dehyd',
 'Glyco_tranf_2_3',
 'Glyco_trans_1_4',
 'Glycos_transf_1',
 'Glycos_transf_2',
 'Hexapep',
 'Isochorismatase',
 'Metallophos',
 'Metallophos_2',
 'NAD_synthase',
 'NTP_transf_5',
 'Peptidase_C25',
 'Peptidase_M73',
 'Polysacc_synt',
 'Polysacc_synt_2',
 'Polysacc_synt_3',
 'Polysacc_synt_C',
 'PqqD (TIGR03859)',
 'TIGR00552',
 'TIGR00679',
 'TIGR01006',
 'TIGR01007',
 'TIGR01099',
 'TIGR01179',
 'TIGR01181',
 'TIGR01188',
 'TIGR01207',
 'TIGR01214',
 'TIGR01479',
 'TIGR01513',
 'TIGR01536',
 'TIGR01575',
 'TIGR01730',
 'TIGR02203',
 'TIGR02204',
 'TIGR02211',
 'TIGR02228',
 'TIGR02789',
 'TIGR02790',
 'TIGR03025',
 'TIGR03026',
 'TIGR03088',
 'TIGR03108',
 'TIGR03522',
 'TIGR03570',
 'TIGR03740',
 'TIGR03796',
 '

In [54]:
dom_cl_rev = {}
for x in dom_cl:
    for y in dom_cl[x]:
        dom_cl_rev[y] = x

In [56]:
clusters_domcl = {}
for x in clusters:
    clusters_domcl[x] = []
    for y in clusters[x]:
        if y in dom_cl_rev:
            dom = dom_cl_rev[y]
        else:
            dom = y
        clusters_domcl[x].append(dom)

In [58]:
counter_domcl = count(clusters_domcl)
cocounter_domcl = cocount(clusters_domcl)

In [60]:
lpp_counter_domcl = count(clusters_domcl)
lpp_cocounter_domcl = cocount(clusters_domcl)

In [62]:
df_domcl = pd.DataFrame.from_dict(counter_domcl, orient='index').reset_index()
df_domcl.columns = ['Domain', 'Count']
df_domcl = df_domcl.sort_values(by='Count', ascending=False)
df_domcl['Description'] = df_domcl['Domain'].map(descriptions)

In [64]:
codf_domcl = pd.DataFrame([(k[0], k[1], v) for k, v in cocounter_domcl.items()])
codf_domcl.columns = ['Domain 1', 'Domain 2', 'Co-occurrence']
codf_domcl = codf_domcl.sort_values(by='Co-occurrence', ascending=False)

In [66]:
with pd.ExcelWriter('counts_domcl.xlsx', engine='xlsxwriter') as writer:
    df_domcl.to_excel(writer, sheet_name='Domain count', index=False)
    codf_domcl.to_excel(writer, sheet_name='Domain co-occurrence', index=False)

# Per connected component

In [65]:
net = pd.read_csv('peptidase sequence pruned default node.csv')

In [69]:
cc_rev = net.set_index('id')['__ccCluster'].to_dict()

In [71]:
cc_rev

{'WP_113034598.1': 1,
 'WP_051395626.1': 1,
 'WP_077614651.1': 1,
 'WP_042234937.1': 1,
 'WP_120190644.1': 1,
 'WP_114345245.1': 1,
 'WP_103996497.1': 1,
 'WP_113022977.1': 1,
 'WP_128658953.1': 1,
 'WP_121610083.1': 1,
 'WP_083938060.1': 1,
 'WP_014653103.1': 1,
 'WP_098445242.1': 1,
 'PFG15025.1': 1,
 'WP_095299855.1': 1,
 'WP_068777003.1': 1,
 'AFC33883.1': 1,
 'WP_051865283.1': 1,
 'WP_056608895.1': 1,
 'WP_009676886.1': 1,
 'WP_108674686.1': 1,
 'WP_129378445.1': 1,
 'WP_013921423.1': 1,
 'WP_098118859.1': 1,
 'WP_091167130.1': 1,
 'WP_057915887.1': 1,
 'WP_090796759.1': 1,
 'WP_083964681.1': 1,
 'OAS85161.1': 1,
 'WP_101566229.1': 1,
 'WP_098846526.1': 1,
 'KML27626.1': 1,
 'WP_065857748.1': 1,
 'WP_014642254.1': 1,
 'WP_051567174.1': 1,
 'WP_121662772.1': 1,
 'WP_056831861.1': 1,
 'KHE71671.1': 1,
 'WP_052158783.1': 1,
 'WP_111614096.1': 1,
 'WP_083537035.1': 1,
 'WP_112880596.1': 1,
 'WP_053074081.1': 1,
 'RBP88261.1': 1,
 'WP_035436624.1': 1,
 'PHA03103.1': 1,
 'WP_052500325.1

In [73]:
cc = {}
for x in cc_rev:
    if cc_rev[x] not in cc:
        cc[cc_rev[x]] = []
    cc[cc_rev[x]].append(x)

In [75]:
cc

{1: ['WP_113034598.1',
  'WP_051395626.1',
  'WP_077614651.1',
  'WP_042234937.1',
  'WP_120190644.1',
  'WP_114345245.1',
  'WP_103996497.1',
  'WP_113022977.1',
  'WP_128658953.1',
  'WP_121610083.1',
  'WP_083938060.1',
  'WP_014653103.1',
  'WP_098445242.1',
  'PFG15025.1',
  'WP_095299855.1',
  'WP_068777003.1',
  'AFC33883.1',
  'WP_051865283.1',
  'WP_056608895.1',
  'WP_009676886.1',
  'WP_108674686.1',
  'WP_129378445.1',
  'WP_013921423.1',
  'WP_098118859.1',
  'WP_091167130.1',
  'WP_057915887.1',
  'WP_090796759.1',
  'WP_083964681.1',
  'OAS85161.1',
  'WP_101566229.1',
  'WP_098846526.1',
  'KML27626.1',
  'WP_065857748.1',
  'WP_014642254.1',
  'WP_051567174.1',
  'WP_121662772.1',
  'WP_056831861.1',
  'KHE71671.1',
  'WP_052158783.1',
  'WP_111614096.1',
  'WP_083537035.1',
  'WP_112880596.1',
  'WP_053074081.1',
  'RBP88261.1',
  'WP_035436624.1',
  'PHA03103.1',
  'WP_052500325.1',
  'WP_010678062.1',
  'WP_025028645.1',
  'WP_114744548.1',
  'WP_051250709.1',
  'WP

In [199]:
def tigr_name(x, descriptions):
    if x.startswith('TIGR'):
        name = descriptions[x].split(': ')[0]
    else:
        name = x
    return name

In [ ]:
def count_filter(filter, clusters):
    flat_clusters = []
    for x in clusters:
        if x in filter:
            flat_clusters += clusters[x]
    return(Counter(flat_clusters))

In [ ]:
def cocount_filter(filter, clusters):
    flat_pairs = []
    for x in clusters:
        if x in filter:
            domains = set(clusters[x])
            pairs = list(combinations(domains, 2))
            flat_pairs += [tuple(sorted(x)) for x in pairs]
    return(Counter(flat_pairs))

In [201]:
count_per_cc = {}
cocount_per_cc = {}
for x in cc:
    counter = count_filter(cc[x], clusters_domcl)
    cocounter = cocount_filter(cc[x], clusters_domcl)

    rename = {}
    n = 0
    for y in counter:
        n += 1
        rename[y] = 'cc' + str(x) + '_' + str(n)
        count_per_cc[rename[y]] = [y, tigr_name(y, descriptions), descriptions[y], x, counter[y], counter[y]/len(cc[x]), counter_domcl[y]]

    for z in cocounter:
        tuple_renamed = tuple(rename[dom] for dom in z)
        cocount_per_cc[tuple_renamed] = [cocounter[z], cocounter[z]/len(cc[x])]

In [203]:
cocount_per_cc

{('cc1_2', 'cc1_8'): [30, 0.23622047244094488],
 ('cc1_1', 'cc1_8'): [7, 0.05511811023622047],
 ('cc1_11', 'cc1_8'): [102, 0.8031496062992126],
 ('cc1_14', 'cc1_8'): [13, 0.10236220472440945],
 ('cc1_16', 'cc1_8'): [1, 0.007874015748031496],
 ('cc1_12', 'cc1_8'): [104, 0.8188976377952756],
 ('cc1_8', 'cc1_7'): [113, 0.889763779527559],
 ('cc1_9', 'cc1_8'): [110, 0.8661417322834646],
 ('cc1_10', 'cc1_8'): [115, 0.905511811023622],
 ('cc1_15', 'cc1_8'): [26, 0.2047244094488189],
 ('cc1_4', 'cc1_8'): [5, 0.03937007874015748],
 ('cc1_6', 'cc1_8'): [17, 0.13385826771653545],
 ('cc1_8', 'cc1_13'): [31, 0.2440944881889764],
 ('cc1_3', 'cc1_8'): [13, 0.10236220472440945],
 ('cc1_5', 'cc1_8'): [27, 0.2125984251968504],
 ('cc1_17', 'cc1_8'): [1, 0.007874015748031496],
 ('cc1_2', 'cc1_1'): [3, 0.023622047244094488],
 ('cc1_2', 'cc1_11'): [29, 0.2283464566929134],
 ('cc1_2', 'cc1_14'): [3, 0.023622047244094488],
 ('cc1_2', 'cc1_16'): [1, 0.007874015748031496],
 ('cc1_2', 'cc1_12'): [29, 0.22834645

In [205]:
count_per_cc

{'cc1_1': ['PS_pyruv_trans',
  'PS_pyruv_trans',
  'Polysaccharide pyruvyl transferase',
  1,
  7,
  0.05511811023622047,
  7],
 'cc1_2': ['Glycos_transf_1',
  'Glycos_transf_1',
  'Glycosyl transferases group 1',
  1,
  44,
  0.3464566929133858,
  66],
 'cc1_3': ['TIGR03570',
  'NeuD_NnaD',
  'NeuD_NnaD: sugar O-acyltransferase, sialic acid O-acetyltransferase NeuD family',
  1,
  13,
  0.10236220472440945,
  15],
 'cc1_4': ['Glyco_hydro_88',
  'Glyco_hydro_88',
  'Glycosyl Hydrolase Family 88',
  1,
  5,
  0.03937007874015748,
  5],
 'cc1_5': ['Polysacc_synt',
  'Polysacc_synt',
  'Polysaccharide biosynthesis protein',
  1,
  28,
  0.2204724409448819,
  31],
 'cc1_6': ['Acyl_transf_3',
  'Acyl_transf_3',
  'Acyltransferase family',
  1,
  22,
  0.1732283464566929,
  23],
 'cc1_7': ['no_match',
  'no_match',
  'No matching domains found',
  1,
  401,
  3.15748031496063,
  661],
 'cc1_8': ['TIGR04249',
  'SCM_chp_ScmC',
  'SCM_chp_ScmC: SynChlorMet cassette protein ScmC',
  1,
  151,
 

In [207]:
df = pd.DataFrame.from_dict(count_per_cc, orient='index').reset_index()
df.columns = ['cc_id', 'domain', 'domain_name', 'description', 'cc', 'cc_count', 'norm_cc_count', 'total_count']
df = df.sort_values(by='total_count', ascending=False)

In [209]:
df

,cc_id,domain,domain_name,description,cc,cc_count,norm_cc_count,total_count
433,cc36_2,no_match,no_match,No matching domains found,36,3,3.000000,661
1035,cc8_1,no_match,no_match,No matching domains found,8,8,4.000000,661
889,cc21_2,no_match,no_match,No matching domains found,21,10,10.000000,661
975,cc5_7,no_match,no_match,No matching domains found,5,31,6.200000,661
1066,cc16_9,no_match,no_match,No matching domains found,16,1,1.000000,661
...,...,...,...,...,...,...,...,...
490,cc4_26,HTH_38,HTH_38,Helix-turn-helix domain,4,1,0.166667,1
506,cc4_42,TIGR03990,Arch_GlmM,Arch_GlmM: phosphoglucosamine mutase,4,1,0.166667,1
507,cc4_43,End_N_terminal,End_N_terminal,N terminal extension of bacteriophage endosial...,4,1,0.166667,1
508,cc4_44,polyprenyl_synt,polyprenyl_synt,Polyprenyl synthetase,4,1,0.166667,1


In [211]:
codf = pd.DataFrame([(k[0], k[1], v[0], v[1]) for k, v in cocount_per_cc.items()])
codf.columns = ['cc_id 1', 'cc_id 2', 'co-occurrence', 'norm_co-occurrence']
codf = codf.sort_values(by='co-occurrence', ascending=False)

In [213]:
codf

,cc_id 1,cc_id 2,co-occurrence,norm_co-occurrence
92,cc1_10,cc1_7,120,0.944882
8,cc1_10,cc1_8,115,0.905512
6,cc1_8,cc1_7,113,0.889764
100,cc1_9,cc1_10,111,0.874016
7,cc1_9,cc1_8,110,0.866142
...,...,...,...,...
5133,cc1_348,cc1_10,1,0.007874
5132,cc1_340,cc1_345,1,0.007874
5131,cc1_348,cc1_340,1,0.007874
5130,cc1_340,cc1_13,1,0.007874


In [215]:
with pd.ExcelWriter('cc_counts.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Domain count', index=False)
    codf.to_excel(writer, sheet_name='Domain co-occurrence', index=False)

In [216]:
df.to_csv('cc_counts.csv', index=False)
codf.to_csv('cc_cocounts.csv', index=False)